<a href="https://colab.research.google.com/github/bhuvana-ak/uplimit_projects/blob/main/uplimitopensourcellmsweekorponotebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install nvidia-ml-py3

  Preparing metadata (setup.py) ... done
  Created wheel for nvidia-ml-py3: filename=nvidia_ml_py3-7.352.0-py3-none-any.whl size=19173 sha256=c910436f30ba9020a4130f03d9203247f1505e5af417746b69707ae6dcc4bce1
  Stored in directory: /root/.cache/pip/wheels/5c/d8/c0/46899f8be7a75a2ffd197a23c8797700ea858b9b34819fbf9e
Successfully built nvidia-ml-py3


In [29]:
import nvidia_smi
def print_gpu_utilization():
  nvidia_smi.nvmlInit()
  handle = nvidia_smi.nvmlDeviceGetHandleByIndex(0)
  # card id 0 hardcoded here, there is also a call to get all available card ids, so we could iterate
  # Inside your training loop
  res = nvidia_smi.nvmlDeviceGetUtilizationRates(handle)
  print(f"GPU utilization: {res.gpu}%, Memory utilization: {res.memory}%")
#nvidia_smi.nvmlShutdown()
print_gpu_utilization()

GPU utilization: 0%, Memory utilization: 0%


In [3]:
# Install required libraries
!pip install -U git+https://github.com/huggingface/transformers -qq
!pip install -U git+https://github.com/huggingface/trl.git -qq
!pip install -U peft -qq
!pip install accelerate -qq
!pip install bitsandbytes -qq
!pip install datasets -qq

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 43.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.8/109.8 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 36.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 7.1 MB/s eta 0:00:00


In [4]:
# Configure Git LFS to handle large files properly
!git lfs install

Git LFS initialized.


In [ ]:


# # Set a higher Git LFS buffer size for large files
# !git config --global http.postBuffer 1048576000

# # Clone with LFS files
# !GIT_LFS_SKIP_SMUDGE=1 git clone https://huggingface.co/uonyeka/uonyeka-llama-3.2.Instruct
# meta-llama/Llama-3.2-1B-Instruct
# %cd uonyeka-llama-3.2.Instruct
# !git lfs pull

Load Dataset and Model , Lora config

In [11]:
import torch
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import LoraConfig, get_peft_model
from trl import ORPOTrainer, ORPOConfig, setup_chat_format


In [6]:
# Load dataset
dataset = load_dataset('mlabonne/orpo-dpo-mix-40k')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/3.53k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/127M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/44245 [00:00<?, ? examples/s]

In [9]:
dataset["train"] = dataset["train"].shuffle(seed=42).select(range(100))

In [12]:
len(dataset["train"])

100

In [10]:


print(dataset["train"][0])
print(dataset["train"][0].keys())
print(dataset["train"][0]['prompt'])
print(dataset["train"][0]['chosen'][0].keys())
print(dataset["train"][0]['chosen'][0]['content'])
print(dataset["train"][0]['rejected'][0]['content'])
print(dataset["train"][0]['question'])




{'source': 'Airoboros', 'chosen': [{'content': "Create a children's book about a young dinosaur enthusiast who travels back in time to study these magnificent creatures firsthand, learning valuable lessons along the way.", 'role': 'user'}, {'content': "Title: Dino Time Traveler\n\nChapter 1: A Budding Paleontologist\n\nOnce upon a time, in the quaint town of Fossilville, there lived a young dinosaur enthusiast named Dexter. He was utterly fascinated by these magnificent creatures and spent hours poring over books about them. His room was adorned with posters, models, and even fossil replicas of his favorite dinosaurs.\n\nDexter's passion for paleontology was ignited when he visited the local museum on a school field trip. The towering skeletons of Triceratops, Stegosaurus, and Tyrannosaurus rex left him awestruck. From that day forward, he dreamt of traveling back in time to study these ancient beasts firsthand.\n\nOne evening, while reading a book on prehistoric life, Dexter stumbled 

In [15]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) Y
Token is valid (permission: write).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in 

In [21]:
!git lfs install


Git LFS initialized.


In [24]:

# Set a higher Git LFS buffer size for large files
!git config --global http.postBuffer 1048576000

# Clone with LFS files
!GIT_LFS_SKIP_SMUDGE=1 git clone https://huggingface.co/uonyeka/uonyeka-llama-3.2.Instruct
%cd uonyeka-llama-3.2.Instruct
!git lfs pull

Cloning into 'uonyeka-llama-3.2.Instruct'...
remote: Enumerating objects: 16, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 16 (delta 2), reused 0 (delta 0), pack-reused 3 (from 1)
Unpacking objects: 100% (16/16), 2.23 MiB | 1.83 MiB/s, done.
/content/uonyeka-llama-3.2.Instruct


In [25]:
# TODO: Choose a base model
# Hint: You can use "EleutherAI/gpt-neo-1.3B" or "openai-community/gpt2" any other suitable model
# model_name = "EleutherAI/gpt-neo-2.7B"
# model_name = "meta-llama/Llama-3.2-1B-Instruct"
model_name = "uonyeka/uonyeka-llama-3.2.Instruct"
# Load model and tokenizer
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# TODO: Set padding token if it doesn't exist
# Hint: Check if tokenizer.pad_token is None, and if so, set it to tokenizer.eos_token
# Also set the corresponding token ID in the model config
# Your code here
# Set padding token
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    model.config.pad_token_id = model.config.eos_token_id

# TODO: Define LoRA configuration
# Hint: Use LoraConfig with appropriate parameters
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

# Apply LoRA to the model
model = get_peft_model(model, lora_config)

config.json:   0%|          | 0.00/913 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

PreProcess Data

In [26]:
# Define max length
max_length = 2048  # or whatever your model's maximum is

# TODO: Implement the preprocess_dataset function
# This function should tokenize the prompt, chosen, and rejected contents
# Hint: Use tokenizer.encode() and tokenizer.decode()
def preprocess_dataset(data_value):
    # Your code here
    tokenizer.truncation_side = "left"  # or "right", depending on your preference

    prompt = tokenizer.encode(data_value['prompt'], truncation=True, max_length=max_length)
    chosen_content = data_value['chosen'][0]['content'] if data_value['chosen'] else ""
    rejected_content = data_value['rejected'][0]['content'] if data_value['rejected'] else ""

    chosen = tokenizer.encode(chosen_content, truncation=True, max_length=max_length)
    rejected = tokenizer.encode(rejected_content, truncation=True, max_length=max_length)

    return {
        "prompt": tokenizer.decode(prompt),
        "chosen": tokenizer.decode(chosen),
        "rejected": tokenizer.decode(rejected)
    }

# Apply the preprocessing to your dataset
preprocessed_dataset = dataset["train"].map(preprocess_dataset)

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [27]:


# TODO: Define ORPO configuration
# Hint: Use ORPOConfig with appropriate training parameters
orpo_config = ORPOConfig(
    learning_rate=1e-5,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    max_steps=100,
    warmup_steps=10,
    gradient_checkpointing=True,
    fp16=True,
    logging_steps=10,
    output_dir="./orpo_output",
    optim="adamw_torch",
    remove_unused_columns=False,
    max_length=max_length,
    max_prompt_length=512,
   #report_to="none",
)

# TODO: Initialize ORPOTrainer
# Hint: Use ORPOTrainer with the model, config, dataset, and tokenizer
trainer = ORPOTrainer(
    model=model,
    args=orpo_config,
    peft_config=lora_config,
    train_dataset=preprocessed_dataset,
    processing_class=tokenizer,
)
# train_dataset=preprocessed_dataset["train"],
# tokenizer=tokenizer,
# Train the model
print_gpu_utilization()
trainer.train()
print_gpu_utilization()

# Save the final model
#trainer.save_model("./final_model")

trainer.save_model("./final_model_llama_3.2.Instruct")
model.save_pretrained("./final_model")
tokenizer.save_pretrained("./final_model")

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


OutOfMemoryError: CUDA out of memory. Tried to allocate 64.00 MiB. GPU 0 has a total capacity of 14.75 GiB of which 61.06 MiB is free. Process 3813 has 14.69 GiB memory in use. Of the allocated memory 14.52 GiB is allocated by PyTorch, and 28.91 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)